In [2]:
## importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path='/content/drive/MyDrive/Malignant Comments Classifier Project/train.csv'
data=pd.read_csv(path)

In [4]:
df_class0 = data.loc[(data['malignant'] == 0) & (data['highly_malignant'] == 0) & (data['rude'] == 0) & (data['threat'] == 0) & (data['abuse'] == 0) & (data['loathe'] == 0)]
df_class1 = data[(data['malignant'] == 1) | (data['highly_malignant'] == 1) | (data['rude'] == 1) | (data['threat'] == 1) | (data['abuse'] == 1) | (data['loathe'] == 1)]

In [5]:
from sklearn.utils import resample
df_class0_downsampled = resample(df_class0, 
                                 replace=False,    
                                 n_samples=16225,     
                                 random_state=42)

In [6]:
df_class0_downsampled

,id,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe
46733,7ce2c22fd3409fbb,"""\n\nOh, don't worry about me, Sandstein. I'm ...",0,0,0,0,0,0
110326,4e366a5778e1b5b4,Are you trying to dispute that fact?,0,0,0,0,0,0
76687,cd62283cfe430f72,SWOT analysis \n\nThis source – Align Technolo...,0,0,0,0,0,0
36231,60cdd48fb71157e6,"cover \n\nso, do we want a current or older co...",0,0,0,0,0,0
21514,38b04f0b55ac0c31,P.S. It's probably worth setting up a template...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
29862,4f4558c105fd8e68,"I dont, however Gun Powder Ma, would you mind ...",0,0,0,0,0,0
90273,f192a8ad369f691f,"""\n\n My Draft of Kaithal honour killing case ...",0,0,0,0,0,0
120264,8339e6cc7623b3f2,tHANX FOR THE TIP ILL DO SO AS SOON AS I CAN!!...,0,0,0,0,0,0
158396,ed23f1af8f561480,"""\n\nTrivia?\n""""Lightfoot is referenced in man...",0,0,0,0,0,0


In [7]:
df=pd.concat([df_class0_downsampled,df_class1])

In [8]:
## Importing more important libraries
from nltk.corpus import wordnet
import string
import nltk
import ast
import re
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [9]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
def clean_text(text):
    text=str(text)
    text = text.lower()
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    rem_num = re.sub('[0-9]+', '', text)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)
df["comment_text"] = df["comment_text"].apply(lambda x: clean_text(x))

In [10]:
comments = df['comment_text'].values
y_list = ["malignant", "highly_malignant", "rude", "threat", "abuse",	"loathe"]
y = df[y_list].values
comments

array(['worry sandstein strong opinion well editing wikipedia personal benefit coi unfinished business articles arbitration sanctions rfar clarification dispute last block ostensibly sanction hope resolved without much fuss hand hoped past well control others respond understand recommendation see well motivated thanks',
       'trying dispute fact',
       'swot analysis source align technology inc swot analysis used times indication find whether primary source assume going remove anything sourced leaving note case objections talk',
       ...,
       'absurd edits absurd edits great white shark total vandalism sexual edit fucking bullshit like spam useful encyclopedia stop bullshit admins everywhere choice stop bullshit else blocked permanently user factualman',
       'hey listen ever delete edits ever annoyed wwe roster confirmed stupid ass deletes write stop please stop work wwe games stop deleting peoples shit get wrong others get wrong let get hang eventually stick ass gonna dele

In [11]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(comments))

In [12]:
from keras.preprocessing import text, sequence

seq = tokenizer.texts_to_sequences(comments)
pad = sequence.pad_sequences(seq, maxlen=100)

In [17]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers


def model_add():
    inputs = Input(shape=(100, ))
    x = Embedding(20000, 128)(inputs)
    x = Bidirectional(LSTM(80))(x)
    x = Dropout(0.1)(x)
    x = Dense(80, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model
model = model_add()
print(model.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 160)               133760    
_________________________________________________________________
dropout_2 (Dropout)          (None, 160)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 80)                12880     
_________________________________________________________________
dropout_3 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                

In [18]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
model.fit(pad, y, batch_size=32, epochs=5, validation_split=0.1, callbacks=early)

Epoch 1/5
913/913 [==============================] - 145s 159ms/step - loss: 0.2139 - accuracy: 0.9404 - val_loss: 0.3442 - val_accuracy: 0.9414
Epoch 2/5
913/913 [==============================] - 145s 159ms/step - loss: 0.1556 - accuracy: 0.9517 - val_loss: 0.3314 - val_accuracy: 0.9411
Epoch 3/5
913/913 [==============================] - 144s 158ms/step - loss: 0.1308 - accuracy: 0.9220 - val_loss: 0.3548 - val_accuracy: 0.9381
Epoch 4/5
913/913 [==============================] - 144s 158ms/step - loss: 0.1105 - accuracy: 0.8208 - val_loss: 0.3981 - val_accuracy: 0.9297
Epoch 5/5
913/913 [==============================] - 144s 158ms/step - loss: 0.0918 - accuracy: 0.7183 - val_loss: 0.4200 - val_accuracy: 0.8508


In [15]:
test_path='/content/drive/MyDrive/Malignant Comments Classifier Project/test.csv'
test = pd.read_csv(test_path)
test = test['comment_text'].values
test_seq = tokenizer.texts_to_sequences(test)
test_pad = sequence.pad_sequences(test_seq, maxlen=100)

In [16]:
y_test = model.predict([test_pad], batch_size=1024, verbose=1)
predicted_submisson= pd.read_csv(test_path)
predicted_submisson[y_list] = y_test
predicted_submisson.to_csv('predicted_submisson.csv', index=False)

150/150 [==============================] - 84s 561ms/step
